# Get gender

## Add `'gender'` key to database

In [1]:
import requests
from IPython.display import JSON
from gender_guesser_first_names import gender
from re import sub
import json

In [2]:
LOCAL=False
if LOCAL:
    f=open('data/inspire_LA.json','r')
    df=json.load(f)
    f.close()
else:
    r=requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df=r.json()

In [3]:
D=gender.Detector(case_sensitive=False)

In [83]:
def force_get_gender(d):
    '''
     Get names from `name.value` and if empty get it from `full_name`
     and using `country` information if avalaible
    '''
    gender = 'unknown'
    names = ''
    if (d.get('name') and d.get('name').get('value') and 
        sub('\w\.','',d.get('name').get('value').split(', ')[-1]).strip()):
        names = sub('\w\.','',d.get('name').get('value').split(', ')[-1]).strip()
    if not names and d.get('full_name'):
        names = sub('\w\.','',d.get('full_name').split(', ')[-1]).strip()
    if names:
        if d.get('country'):
            gender = D.get_gender( get_names(d), country=d.get('country').strip() )
        else:
            gender = D.get_gender( get_names(d) )
    return gender

In [86]:
kk = [d.update( {'gender': force_get_gender(d)} ) for d in df ]

In [87]:
f=open('inspire_LA_gender.json','w')
json.dump(df,f)
f.close()

## Analysis

In [88]:
import pandas as pd

In [89]:
DF = pd.read_json('inspire_LA_gender.json')

Unique authors ids

In [90]:
DF.shape[0]

57588

In [91]:
DF = DF.drop_duplicates('author_id').reset_index(drop=True)
DF.shape[0]

29969

In [92]:
DF['first_names'] = DF['name'].str.get('value').astype(str).str.split(', ').str[-1]#.apply(
    #lambda s: sub('\w\.','',s)).str.replace('None','')

In [93]:
DF['gender'].unique()

array(['male', 'unknown', 'female', 'andy'], dtype=object)

In [94]:
print(DF['gender'].shape[0])
total = DF['gender'].dropna().shape[0]
total

29969


29969

In [95]:
male = DF[DF['gender'] == 'male'].shape[0]
female = DF[DF['gender'] == 'female'].shape[0]
unknown = DF[DF['gender'] == 'unknown'].shape[0]
unknown = unknown + DF[DF['gender'] == 'andy'].shape[0]

In [96]:
assert male + female + unknown == total

In [97]:
male+female

17335

In [98]:
male

14626

In [99]:
female

2709

In [100]:
round(female/male,2)

0.19

In [101]:
unknown

12634

In [102]:
DFFM = DF[ (( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFFM.shape

(17335, 14)

In [103]:
DFU = DF[ ~(( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFU.shape

(12634, 14)

In [104]:
DFFM[['first_names','country','gender']].sample(50)

,first_names,country,gender
11252,Daniel Victor,Italy,male
8720,Cristian A.,Chile,male
20796,Andrea,Italy,male
14,Anamaria,Venezuela,female
20624,Rafael,Brazil,male
1942,Angel Giovanni,Guatemala,male
13422,Jorge,United States,male
4799,Robert.L.,United States,male
20677,Denis,Brazil,male
6649,Rogerio,Brazil,male


In [105]:
#[['first_names','country','gender']]
DFU.sample(50)[['author_id','full_name','name','country','gender']]

,author_id,full_name,name,country,gender
14804,J.L.Mateos.1,"Mateos, J.L.",None,Mexico,unknown
4423,M.Cappellari.1,"Cappellari, M.","{'value': 'Cappellari, M.', 'preferred_name': ...",Netherlands,unknown
27767,A.C.Rodrigues.Mendes.1,"Rodrigues Mendes, A.C.",None,Brazil,unknown
15463,P.D.Miller.1,"Miller, P.D.",None,United States,unknown
26366,A.L.R.Dos.santos.1,"dos Santos, A. L. R.",None,Brazil,unknown
12742,M.R.Zirnbauer.1,"Zirnbauer, M.R.","{'value': 'Zirnbauer, M.R.', 'preferred_name':...",United States,unknown
17259,C.Oropeza.1,"Oropeza, C.",None,Mexico,unknown
9599,K.Aryal.2,"Aryal, Krishna",None,United States,andy
29922,A.Tenorio.1,"Tenorio, A.",None,Brazil,unknown
3766,F.X.Josse.Michaux.1,"Josse-Michaux, Francois-Xavier","{'value': 'Josse-Michaux, Francois-Xavier', 'p...",France,unknown
